<a href="https://colab.research.google.com/github/GuevaraMarcos/TelecomX/blob/main/Telecom%20X%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis Predictivo de Rotación de Clientes - Versión Completa y Visualmente Optimizada

1. Configuración Inicial y Carga de Datos

1.1. Librerías y Configuración Visual

In [ ]:
# Módulos esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                            f1_score, roc_auc_score, confusion_matrix,
                            ConfusionMatrixDisplay, classification_report)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib
import warnings
warnings.filterwarnings('ignore')

# Paleta de colores corporativa
COLORES = {
    'azul': '#2E86AB',
    'rojo': '#C73E1D',
    'verde': '#3A7D44',
    'naranja': '#F18F01',
    'gris': '#333333',
    'fondo': '#F7F7F7'
}

# Configuración global de estilos
plt.style.use('seaborn-whitegrid')
sns.set_palette(list(COLORES.values()))
plt.rcParams.update({
    'figure.facecolor': COLORES['fondo'],
    'axes.facecolor': COLORES['fondo'],
    'axes.edgecolor': COLORES['gris'],
    'axes.labelcolor': COLORES['gris'],
    'text.color': COLORES['gris'],
    'xtick.color': COLORES['gris'],
    'ytick.color': COLORES['gris']
})

1.2. Carga y Preprocesamiento de Datos

In [ ]:
def cargar_datos(ruta):
    df = pd.read_csv(ruta)

    # Eliminar ID único
    df = df.drop(['customerid'], axis=1)

    # Convertir booleanos a enteros
    bool_cols = ['customer_partner', 'customer_dependents', 'account_paperlessbilling']
    df[bool_cols] = df[bool_cols].astype(int)

    # Nueva feature: ratio de gasto mensual/total
    df['ratio_gasto'] = df['account_charges_monthly'] / (df['account_charges_total'] + 1e-6)

    # Nueva feature: antigüedad ponderada por servicios
    df['antiguedad_ponderada'] = df['customer_tenure'] * df['total_services']

    return df

# Cargar datos
datos = cargar_datos('datos_tratados_telecomX.csv')

2. Análisis Exploratorio (EDA) con Visualizaciones Avanzadas

2.1. Distribución de Variables Clave

In [ ]:
def eda_visual(df):
    fig, ax = plt.subplots(2, 2, figsize=(16, 12))

    # Gráfico 1: Distribución de antigüedad (KDE)
    sns.kdeplot(df['customer_tenure'], fill=True, color=COLORES['azul'], ax=ax[0,0])
    ax[0,0].set_title('Distribución de Antigüedad (meses)', fontsize=12)
    ax[0,0].axvline(df['customer_tenure'].mean(), color=COLORES['rojo'], linestyle='--')

    # Gráfico 2: Churn por tipo de contrato (Barras apiladas)
    pd.crosstab(df['account_contract'], df['churn']).plot(
        kind='bar', stacked=True,
        color=[COLORES['azul'], COLORES['rojo']],
        ax=ax[0,1]
    )
    ax[0,1].set_title('Rotación por Tipo de Contrato', fontsize=12)

    # Gráfico 3: Boxplot de cargos mensuales vs churn
    sns.boxplot(x='churn', y='account_charges_monthly', data=df,
                palette=[COLORES['azul'], COLORES['rojo']], ax=ax[1,0])
    ax[1,0].set_title('Cargos Mensuales vs Rotación', fontsize=12)

    # Gráfico 4: Correlación entre variables numéricas (Heatmap)
    corr = df.select_dtypes(include=np.number).corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, ax=ax[1,1])
    ax[1,1].set_title('Matriz de Correlación', fontsize=12)

    plt.tight_layout()
    plt.show()

eda_visual(datos)

2.2. Análisis de Desbalanceo de Clases

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='churn', data=datos, palette=[COLORES['azul'], COLORES['rojo']])
plt.title('Distribución de Churn (0=No, 1=Sí)', fontsize=14)
plt.show()

print("Proporción de clases:")
print(datos['churn'].value_counts(normalize=True))

3. Preparación de Datos para Modelado

3.1. División Train-Test y Pipeline de Preprocesamiento

In [ ]:
# Definir variables
X = datos.drop('churn', axis=1)
y = datos['churn']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Columnas para transformación
num_cols = ['customer_tenure', 'account_charges_monthly', 'ratio_gasto', 'antiguedad_ponderada']
cat_cols = ['customer_gender', 'account_contract', 'account_paymentmethod']

# Pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), num_cols),
        ('cat', OneHotEncoder(drop='first'), cat_cols)
    ],
    remainder='passthrough'
)

# Pipeline completo con SMOTE para balanceo
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

4. Modelado y Optimización

4.1. Entrenamiento y Evaluación de Modelos

In [ ]:
# Definición de modelos y parámetros
modelos = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'classifier__n_estimators': [100, 200],
            'classifier__max_depth': [5, 10, None],
            'classifier__min_samples_split': [2, 5]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, eval_metric='logloss'),
        'params': {
            'classifier__learning_rate': [0.01, 0.1],
            'classifier__max_depth': [3, 5],
            'classifier__subsample': [0.8, 1.0]
        }
    }
}

# Entrenamiento y evaluación
resultados = {}
for nombre, config in modelos.items():
    pipeline.set_params(classifier=config['model'])

    grid = GridSearchCV(
        pipeline,
        config['params'],
        cv=5,
        scoring='f1',
        n_jobs=-1,
        verbose=1
    )

    grid.fit(X_train, y_train)
    y_pred = grid.best_estimator_.predict(X_test)

    resultados[nombre] = {
        'model': grid.best_estimator_,
        'metrics': {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred)
        },
        'best_params': grid.best_params_
    }

    print(f"\n🔹 Resultados para {nombre}:")
    print(f"Mejores parámetros: {grid.best_params_}")
    print(classification_report(y_test, y_pred))

4.2. Visualización de Resultados

In [ ]:
# Dataframe comparativo
df_resultados = pd.DataFrame.from_dict(
    {k: v['metrics'] for k, v in resultados.items()},
    orient='index'
)

# Gráfico de comparación
df_resultados.plot(kind='bar', figsize=(12, 6), color=[COLORES['azul'], COLORES['verde'], COLORES['naranja'], COLORES['rojo']])
plt.title('Comparación de Modelos', fontsize=14)
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.legend(loc='lower right')
plt.show()

# Matriz de confusión del mejor modelo
mejor_modelo = resultados['XGBoost']['model']
ConfusionMatrixDisplay.from_estimator(mejor_modelo, X_test, y_test, cmap='Blues')
plt.title('Matriz de Confusión - XGBoost')
plt.show()

5. Interpretación y Despliegue

5.1. Importancia de Características

In [ ]:
# Obtener nombres de features
cat_encoder = mejor_modelo.named_steps['preprocessor'].named_transformers_['cat']
cat_features = cat_encoder.get_feature_names_out(cat_cols)
all_features = num_cols + list(cat_features) + [col for col in X.columns if col not in num_cols+cat_cols]

# Importancia para XGBoost
if hasattr(mejor_modelo.named_steps['classifier'], 'feature_importances_'):
    importancia = mejor_modelo.named_steps['classifier'].feature_importances_
    df_importancia = pd.DataFrame({'Feature': all_features, 'Importance': importancia})
    df_importancia = df_importancia.sort_values('Importance', ascending=False).head(10)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=df_importancia, palette='viridis')
    plt.title('Top 10 Características Más Importantes', fontsize=14)
    plt.show()

5.2. Serialización del Modelo

In [ ]:
# Guardar modelo y metadatos
paquete_modelo = {
    'model': mejor_modelo,
    'features': all_features,
    'metrics': resultados['XGBoost']['metrics'],
    'version': '1.0'
}

try:
    joblib.dump(paquete_modelo, 'modelo_churn_telecom.pkl')
    print("✅ Modelo guardado correctamente")
except Exception as e:
    print(f"❌ Error al guardar: {str(e)}")

6. Dashboard de Monitoreo (Simulación)

In [ ]:
def crear_dashboard():
    fig = plt.figure(figsize=(18, 10), facecolor=COLORES['fondo'])
    fig.suptitle('Panel de Monitoreo - Predicción de Churn', fontsize=16)

    # KPI 1: Tasa de rotación
    ax1 = plt.subplot2grid((3, 3), (0, 0))
    ax1.text(0.5, 0.7, "27.3%", fontsize=32, ha='center', color=COLORES['rojo'])
    ax1.text(0.5, 0.4, "Tasa de Rotación", fontsize=14, ha='center')
    ax1.axis('off')

    # KPI 2: Exactitud del modelo
    ax2 = plt.subplot2grid((3, 3), (0, 1))
    ax2.text(0.5, 0.7, "82.1%", fontsize=32, ha='center', color=COLORES['verde'])
    ax2.text(0.5, 0.4, "Exactitud del Modelo", fontsize=14, ha='center')
    ax2.axis('off')

    # Gráfico 3: Tendencia semanal
    ax3 = plt.subplot2grid((3, 3), (1, 0), colspan=2)
    ax3.plot(['Lun', 'Mar', 'Mié', 'Jue', 'Vie'], [45, 52, 60, 48, 55], marker='o', color=COLORES['azul'])
    ax3.set_title('Clientes en Riesgo (Últimos 5 días)')
    ax3.grid(True, linestyle='--', alpha=0.6)

    # Gráfico 4: Distribución de riesgo
    ax4 = plt.subplot2grid((3, 3), (0, 2), rowspan=2)
    ax4.pie([35, 50, 15], labels=['Alto', 'Moderado', 'Bajo'],
            colors=[COLORES['rojo'], COLORES['naranja'], COLORES['verde']],
            autopct='%1.1f%%')
    ax4.set_title('Distribución de Riesgo')

    # Alertas
    ax5 = plt.subplot2grid((3, 3), (2, 0), colspan=3)
    ax5.text(0.1, 0.8, "⚠ Aumento del 15% en clientes riesgo alto", color=COLORES['rojo'], fontsize=12)
    ax5.text(0.1, 0.5, "ℹ Nuevo patrón detectado en clientes con >2 servicios", color=COLORES['azul'], fontsize=12)
    ax5.axis('off')

    plt.tight_layout()
    plt.show()

crear_dashboard()